The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

Visually, if we omit the "pixel" prefix, the pixels make up the image like this:

000 001 002 003 ... 026 027
028 029 030 031 ... 054 055
056 057 058 059 ... 082 083
 |   |   |   |  ...  |   |
728 729 730 731 ... 754 755
756 757 758 759 ... 782 783 

The test data set, (test.csv), is the same as the training set, except that it does not contain the "label" column.

Your submission file should be in the following format: For each of the 28000 images in the test set, output a single line containing the ImageId and the digit you predict. For example, if you predict that the first image is of a 3, the second image is of a 7, and the third image is of a 8, then your submission file would look like:

ImageId,Label
1,3
2,7
3,8 
(27997 more lines)

The evaluation metric for this contest is the categorization accuracy, or the proportion of test images that are correctly classified. For example, a categorization accuracy of 0.97 indicates that you have correctly classified all but 3% of the images.

conda install -c fastai fastai

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import matplotlib.pyplot as plt
#from torchvision import transforms

from fastai.vision.all import *
#https://www.kaggle.com/hongpeiyi/cnn-with-pytorch-and-fastai

for dirname, _, filenames in os.walk('./digit-recognizer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Проверка GPU

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))

input_path = Path("./digit-recognizer/")
train = pd.read_csv(input_path/"train.csv")

class NumberDataset:
    def __init__(self, data, transform=None):
        self.data = data.reset_index(drop=True)
        self.trans = transform
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        
        d = self.data.iloc[index, 1:].values.reshape(1, 28, 28)
        
        if self.trans:
            d = self.trans(tensor(d))
        else:
            d = tensor(d)/255
        
        y = self.data.iloc[index, 0]
        
        return d, tensor(y)
    
train_idx, valid_idx = RandomSplitter()(train)

train_ds = NumberDataset(train.iloc[train_idx, :])
valid_ds = NumberDataset(train.iloc[valid_idx, :])

train_dl = DataLoader(train_ds, bs=64)
valid_dl = DataLoader(valid_ds, bs=128)

dls = DataLoaders(train_dl, valid_dl).to(device)

#test = pd.read_csv('./digit-recognizer/test.csv')

#test = np.array(test, dtype=np.float32)/255
#test = test.reshape(-1,28,28,1)

#x_digit = train.drop(['label'], axis=1)
#x_digit = np.array(x_digit, dtype=np.float32)/255
#x_digit = x_digit.reshape(-1,28,28,1)

#y_digit = train['label']
#y_digit = keras.utils.to_categorical(y_digit, num_classes=10)

#for i in range(100):
#    fig = plt.figure()
#    fig.suptitle(y_digit[i+1])
#    plt.imshow(x_digit[i+1], cmap='gray')

#print(x_digit.shape)



./digit-recognizer/sample_submission.csv
./digit-recognizer/train.csv
./digit-recognizer/test.csv
Using device: cuda
NVIDIA GeForce GT 1030


In [ ]:

trainloader = torch.utils.data.DataLoader(x_digit, batch_size=64, 
                                          shuffle=True, num_workers=4)

# Определение нейронной сети

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # Конструктор суперкласса
        
        # Описание слоёв:
        self.conv1 = nn.Conv2d(3, 6, 5)       #
        self.pool = nn.MaxPool2d(2, 2)        #
        self.conv2 = nn.Conv2d(6, 16, 5)      #
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # Полносвязный слой
        self.fc2 = nn.Linear(120, 84)         # Полносвязный слой
        self.fc3 = nn.Linear(84, 10)          # Полносвязный слой

    # Функция, описывающая поток данных при обучении и предсказании
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)    # Преобразование в одномерный тензор
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Загрузка параметров предобученной модели из файла
PATH = './cifar_net_66pc.pth'
net.load_state_dict(torch.load(PATH))

# Отправляем сеть на GPU
net.to(device)

# Определим функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()  # Функция потерь. 
# Оптимизатор SGD - Стохастический градиентный оптимизатор. 
learn_rate = 0.02
optimizer = optim.SGD(net.parameters(), lr=learn_rate, momentum=0.9) 

start_time = time.time()

x = []
y = []
j = 0

for epoch in range(1):  # loop over the dataset multiple times
    # затухание скорости обучения (learning rate decay)
    if epoch == 2:
        learn_rate = 0.01
    if epoch == 4:
        learn_rate = 0.002
    if epoch == 5:
        learn_rate = 0.0002
    print("Learning rate =", learn_rate)
    optimizer = optim.SGD(net.parameters(), lr=learn_rate, momentum=0.9)

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 20 mini-batches
            x.append(j)
            y.append(running_loss / 20)
            j += 1            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

print("Working time: %s seconds" % (time.time() - start_time))

print('Кривая обучения:')

plt.plot(x, y)
plt.show()



print('Finished Training')

print("Working time: %s seconds" % (time.time() - start_time))

print('Кривая обучения:')

plt.plot(x, y)
plt.show()